In [28]:
import pandas as pd
import requests
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("credentials.json", scope)
client = gspread.authorize(creds)

spreadsheet_url = "https://docs.google.com/spreadsheets/d/1g7uVhRn1czV_kSbaXmjwr-88sxjahTDigl9htQ0xoBw/edit#gid=0"
temp_url = "https://docs.google.com/spreadsheets/d/1vnxBib84T24Hq0BfzWJQobDFaNnmb2A6s36TbLEqrFY/edit?gid=0#gid=0"
holiday_url = "https://docs.google.com/spreadsheets/d/1EZvUyp8vKcgSDke7iclm9b54Sw2JxCG6VzLvGq93XpU/edit?gid=1209652816#gid=1209652816"

spreadsheet = client.open_by_url(spreadsheet_url)
temp = client.open_by_url(temp_url)
holiday = client.open_by_url(holiday_url)

worksheet = spreadsheet.get_worksheet(0)
tempsheet = temp.get_worksheet(0)
holiday = holiday.get_worksheet(0)

data = worksheet.get_all_records()
tdata = tempsheet.get_all_records()
hdata = holiday.get_all_records()

train_data = pd.DataFrame(data)
temp_df = pd.DataFrame(tdata)
holiday_df = pd.DataFrame(hdata)

train_data['sale_date'] = pd.to_datetime(train_data['sale_date'], errors='coerce')

if train_data['sale_date'].isnull().sum() > 0:
    print("Warning: Some dates could not be converted. Check for incorrect formats.")

train_data['ปี'] = train_data['sale_date'].dt.strftime('%Y')
train_data['เดือน'] = train_data['sale_date'].dt.strftime('%m')
train_data['เดือน-ปี'] = train_data['sale_date'].dt.strftime('%m-%Y')
train_data['เดือน-วัน'] = train_data['sale_date'].dt.strftime('%m-%d')


train_data = pd.merge(train_data, holiday_df, how='left', left_on='เดือน-วัน', right_on='วันที่')
train_data['holiday'] = train_data['วันสำคัญ'].notnull()
train_data = pd.merge(train_data, temp_df, how='left', left_on='เดือน-ปี', right_on='เดือนอุณหภูมิ')

rainfall_data = {}
for year in range(2019, 2025):
    url = f"https://data.tmd.go.th/api/ThailandMonthlyRainfall/v1/index.php?uid=api&ukey=api12345&format=json&year={year}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        yearly_rainfall = {}
        for station in data.get('StationMonthlyRainfall', []):
            province = station['StationNameThai']
            rainfall = station['MonthlyRainfall']
            yearly_rainfall[province] = {str(i+1).zfill(2): float(rainfall[f'Rainfall{month}']) for i, month in enumerate(
                ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']
            )}
        rainfall_data[year] = yearly_rainfall
    else:
        print(f"Failed to fetch rainfall data for {year}")

province = "นครราชสีมา"

train_data['ปริมาณน้ำฝน'] = train_data.apply(lambda row: rainfall_data.get(int(row['ปี']), {}).get(province, {}).get(row['เดือน'], 0.0), axis=1)
train_data.drop(columns=['เดือน', 'ปี'], inplace=True)
train_data = train_data.drop(columns=['เดือนอุณหภูมิ'])
train_data = train_data.drop(columns=['เดือน-ปี'])
train_data = train_data.drop(columns=['เดือน-วัน'])
train_data = train_data.drop(columns=['วันที่'])
train_data.to_csv("cleaned_train_data.csv", index=False)

def updateToGSheetFromCSV(csv_file_path):
    datacleaned_url = "https://docs.google.com/spreadsheets/d/1eohhRmXeB-e7xpoacjfbRIA2Igz8GP7A1Sbe3ekqAdQ/edit?gid=0#gid=0"
    datacleaned_spreadsheet = client.open_by_url(datacleaned_url)
    datacleaned_worksheet = datacleaned_spreadsheet.get_worksheet(0)
    
    data_from_csv = pd.read_csv(csv_file_path)
    data_from_csv = data_from_csv.fillna('')
    datacleaned_worksheet.clear()
    datacleaned_worksheet.update('A1', [data_from_csv.columns.tolist()] + data_from_csv.values.tolist())
    
    print("Data successfully uploaded to Google Sheets.")

# train_data.to_csv("cleaned_train_data.csv", index=False)

# Now upload the CSV to Google Sheets
updateToGSheetFromCSV("cleaned_train_data.csv")
train_data

C:\Users\Admin\AppData\Local\Temp\ipykernel_16020\4108184628.py:92: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  datacleaned_worksheet.update('A1', [data_from_csv.columns.tolist()] + data_from_csv.values.tolist())


Data successfully uploaded to Google Sheets.


In [34]:
import pandas as pd

df = train_data
df['sale_date'] = pd.to_datetime(df['sale_date'], format='%Y-%m-%d')

df_grouped = df.groupby(['sale_date', 'drug_code'], as_index=False)['quantity'].sum()
df_grouped = df_grouped.merge(df[['sale_date', 'holiday', 'อุณหภูมิ', 'ปริมาณน้ำฝน']].drop_duplicates(), on='sale_date', how='left')
df_grouped = df_grouped.sort_values(by='sale_date')

df_grouped.to_csv('processed_sales_data.csv', index=False)
df_grouped

In [60]:
name = "O0047"

In [62]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense , Input
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

df = df_grouped

df['วันที่ขาย'] = pd.to_datetime(df['sale_date'])
df = df.sort_values(by='sale_date')

df_drug = df[df['drug_code'] == name]

df_drug = df_drug.copy() 

scaler = MinMaxScaler()
df_drug.loc[:, ['quantity', 'อุณหภูมิ', 'ปริมาณน้ำฝน']] = scaler.fit_transform(df_drug[['quantity', 'อุณหภูมิ', 'ปริมาณน้ำฝน']])

def create_sequence(data, seq_length=30):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data.iloc[i:i+seq_length][['quantity', 'อุณหภูมิ', 'ปริมาณน้ำฝน']].values)
        y.append(data.iloc[i+seq_length]['quantity'])
    return np.array(X), np.array(y)

X, y = create_sequence(df_drug, seq_length=30)
X_train, y_train = X, y

model = Sequential([
    Input(shape=(X_train.shape[1], X_train.shape[2])),  # Define input shape here
    LSTM(units=50, return_sequences=False),
    Dense(units=1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, y_train, epochs=100, batch_size=8)

last_30_days = df_drug.tail(30)[['quantity', 'อุณหภูมิ', 'ปริมาณน้ำฝน']].values

predictions_2021 = []
for i in range(365):
    last_30_days_reshaped = np.reshape(last_30_days, (1, last_30_days.shape[0], last_30_days.shape[1]))
    
    predicted_quantity = model.predict(last_30_days_reshaped)
    
    predictions_2021.append(predicted_quantity[0][0])
    
    last_30_days = np.vstack([last_30_days[1:], np.array([predicted_quantity[0][0], last_30_days[0, 1], last_30_days[0, 2]])])

predictions_2021 = scaler.inverse_transform(np.concatenate((np.array(predictions_2021).reshape(-1, 1), np.zeros((len(predictions_2021), 2))), axis=1))[:, 0]

dates_2021 = pd.date_range(start='2021-01-01', end='2021-12-31', freq='D')

predictions_df = pd.DataFrame({'date': dates_2021, 'predicted_quantity': predictions_2021})

print(predictions_df.head(20))

predictions_df.to_csv('predictions_2021_A0158.csv', index=False)
predictions_df['date'] = pd.to_datetime(predictions_df['date']).dt.strftime('%Y-%m-%d')
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
client = gspread.authorize(creds)

spreadsheet_id = "13_f_4pEUgCifgGYRzME0eFNRVrt_vGMKEcygE-MrFJ8"
spreadsheet = client.open_by_key(spreadsheet_id)

try:
    sheet = spreadsheet.worksheet(name)
    spreadsheet.del_worksheet(sheet) 
except gspread.WorksheetNotFound:
    pass  

sheet = spreadsheet.add_worksheet(title=name, rows=str(len(predictions_df)+1), cols="2")

sheet.update('A1', [['date', 'predicted_quantity']])

sheet.update('A2', predictions_df.values.tolist())

print(f"Predictions saved to Google Sheets successfully!")

Epoch 1/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 0.0985
Epoch 2/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.1542
Epoch 3/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.1171
Epoch 4/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.1405
Epoch 5/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.1439
Epoch 6/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.1011
Epoch 7/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.1016
Epoch 8/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0740
Epoch 9/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0979 
Epoch 10/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0766 
Epoch 11/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.1254
Epoch 12/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.1071
Epoch 13/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.1119
Epoch 14/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1240 
Epoch 15/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1021 
Epoch 16/100
8/8 ━━

C:\Users\Admin\AppData\Local\Temp\ipykernel_16020\1210854340.py:79: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update('A1', [['date', 'predicted_quantity']])
C:\Users\Admin\AppData\Local\Temp\ipykernel_16020\1210854340.py:81: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update('A2', predictions_df.values.tolist())


Predictions saved to Google Sheets successfully!


In [46]:
drug_counts = df.groupby('drug_code').size().reset_index(name='row_count')

print("รายการที่สามารถเทรนได้:")
print(drug_counts)

insufficient_data = drug_counts[drug_counts['row_count'] < 30]

if insufficient_data.empty:
    print("เทรนได้หมด")
else:
    print("รายการที่ไม่สามารถเทรนได้:")
    print(insufficient_data)

Row counts for each drug code:
   drug_code  row_count
0      A0157         17
1      A0158        379
2      A0159         36
3      A0160         69
4      A0167         72
..       ...        ...
93     O0047         92
94     R0078         55
95     S0125         69
96     T0181         56
97   TH-0292          3

[98 rows x 2 columns]
⚠️ These drug codes have fewer than 30 rows:
   drug_code  row_count
0      A0157         17
5      A0168          6
10     A0179          4
12     A0190         29
13     A0191         18
17     B0074         29
18     B0075          1
19     B0080         10
20     B0082          5
22     B0088         16
27     B0111         20
28     B0112         28
29     B0114          5
30     B0134         11
32     B0138         25
33     B0140         15
35     C0125         15
37     C0131         29
38     C0137         12
39     C0138         10
41     C0157         14
43     C0184         17
50     D0169         11
51     D0170          1
61     D0202 